In [1]:
!guardrails hub install hub://guardrails/regex_match --quiet

Installing hub://guardrails/regex_match...
✅Successfully installed guardrails/regex_match!




# Validating Responses using Regex

This notebook demonstrates how to synthesize regex-validated responses from an LLM. 

In this example, we'll ask GPT-3.5 to give us a phone number, and make sure it's just separated by dashes - so something of the format 123-456-7890.


### Define the RegEx

In [2]:
regex = '\\d{3}-\\d{3}-\\d{4}'

### Import Guardrails and openai

In [3]:
from guardrails import Guard
from guardrails.hub import RegexMatch
from rich import print
import os

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

### Create the guard and prompt

We use a simple string guard with the RegexMatch validator. We can see that the initial response is malformatted, but the reask gets the LLM to respond correctly.

In [21]:
guard = Guard.from_string(
    validators=[RegexMatch(regex=regex, match_type='search', on_fail='reask')]
)

guard(
    prompt='I am writing a movie and need a fake phone number. Generate a fake phone number. Do not write anything other than the phone number.',
    model="gpt-4-turbo",
)

print(guard.history.last.tree)

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │ I am writing a movie and need a fake phone number. Generate a fake phone number. Do not write anything  │ │
│   │ │ other than the phone number.                                                                            │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
│   │ │ No message history.                                                                                     │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
│   │ │ (555) 012-3456                                                                                          │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
│   │ │ FieldReAsk(                                                                                             │ │
│   │ │     incorrect_value='(555) 012-3456',                                                                   │ │
│   │ │     fail_results=[                                                                                      │ │
│   │ │         FailResult(                                                                                     │ │
│   │ │             outcome='fail',                                                                             │ │
│   │ │             error_message='Result must match \\d{3}-\\d{3}-\\d{4}',                                     │ │
│   │ │             fix_value='ah300-632-4603ah',                                                               │ │
│   │ │             error_spans=None,                                                                           │ │
│   │ │             metadata=None,                                                                              │ │
│   │ │             validated_chunk=None                                                                        │ │
│   │ │         )                                                                                               │ │
│   │ │     ],                                                                                                  │ │
│   │ │     additional_properties={},                                                                           │ │
│   │ │     path=None                                                                                           │ │
│   │ │ )                                                                                                       │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
└── ╭────────────────────────────────────────────────── Step 1 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │ This was a previous response you generated:                                                             │ │
    │ │                                                                                                         │ │
    │ │ ======                                                                                                  │ │
    │ │ (555) 012-3456                             